# Assignment 1: The 8-puzzle
## CS152 // Prof Shekhar
### Soren Gran // October 2019

<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

### Author: Soren Gran

In [140]:
# Import any packages you need here
# Also define any variables as needed
import numpy as np
import random
# YOUR CODE HERE (OPTIONAL)

#Now, define the class PuzzleNode:
class PuzzleNode(object):
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    
    """
    def __init__(self, state, parent_g, h_score, parent_path):
        self.state = state # first and foremost, each puzzle node is represented by its state
        # (i.e. the arrangement of its tiles)
        self.g_score = parent_g + 1 # in other A* search algorithms, the g_score would change from node to node.
        # However, every move has the same step length in 8 puzzle: 1. So we just add 1.
        self.h_score = h_score # calculated by the heuristic specified by solvePuzzle()
        self.f_score = self.g_score + self.h_score # f is sum of g and h
        self.path = parent_path # the root node starts with an empty list as the parent_path
        self.path.append(state) # we just add the current state onto the parent's path
    
    def __str__(self): # short method to display the grid
        for row in self.state:
            print(row) # print each row
        print('\n') # print a new line in case we are printing repeated states
        # which would look confusing if not for the new line

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [42]:
# Add any additional code here (e.g. for the memoization extension)

# YOUR CODE HERE (OPTIONAL)

# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    state = [item for sublist in state for item in sublist] # this line of code taken from https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
    # this line is designed to flatten the list, making it easier to assess the misplaced tiles
    h = 0
    for index in range(len(state)): # for every tile
        if state[index] != 0: # we do not want to consider the 0 tile, because it actually represents the absence of a tile
            if state[index] != index: # if the tile is not equal to the index (which is its home position),
                h += 1 # we count it as misplaced
    return h

# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # YOUR CODE HERE
    # the Manhattan distance is the total distance of all tiles from their home square
    # we calculate this distance by calculating each tiles' distance and summing them
    # The distance is calculated by subtracting the current position from the ideal positions.
    # For example, if the 1 tile is at position (2, 2) (i.e. the last tile in the last row),
    # we subtract those coordinates from the home coordinates (1, 0) giving us a Manhattan distance of 3
    h = 0
    sides = len(state) # gets us the dimensions of the board
    for row in range(sides): # iterating over 
        y_coord = row # the y_coordinate is the vertical position, or the row number
        for index in range(sides):
            point = state[row][index]
            if point != 0: # again, we do not want to consider the empty space
                x_coord = index # the x_coordinate is the horizontal position, or the index in whichever row
                y_coord_ideal = point // sides # floor division gives us the correct row
                x_coord_ideal = point % sides # modulo division gives us the correct index
                man_dist = abs(y_coord - y_coord_ideal) + abs(x_coord - x_coord_ideal) # add the distances
                h += man_dist # add the distance to the sum
    return h
    
# Extra heuristic for the extension.  If implemented, modify the function below
# This heuristic is a Pattern Database.
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    
    return 0

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [212]:
# Import any packages or define any helper functions you need here

# YOUR CODE HERE (OPTIONAL)
import queue as q
import copy

def swap_positions(state, pos1_a, pos1_b, pos2_a, pos2_b): # this function was taken from
    # https://www.geeksforgeeks.org/python-program-to-swap-two-elements-in-a-list/, although
    # I edited it to work for a list of lists
    # This function is used to "move" a tile to the empty spot
    # in reality, we just switch the 0 tile with the other tile
    state[pos1_a][pos1_b], state[pos2_a][pos2_b] = state[pos2_a][pos2_b], state[pos1_a][pos1_b]
    return state

def get_kids(state):
    # For any board position, this function gets its child nodes
    # It does this by finding the 0 tile (the empty space), identifying its neighbors,
    # and swapping the 0 with each neighbor, returning as many kids as there are neighbors
    sides = len(state)
    kids = []
    for row in range(sides):
        for index in range(sides):
            if state[row][index] == 0: # we have found the 0 tile
                # now we need to get its neighbors
                neighbors = [(row+1, index), (row-1, index), (row, index+1), (row, index-1)]
                for coord_pair in neighbors:
                    neigh_row, neigh_index = coord_pair
                    if -1 < neigh_row < sides:
                        if -1 < neigh_index < sides:
                            # we must make sure we are not getting a neighbor that is off the board
                            # all neighbors must be on the board, and the board does not have periodic boundaries.
                            # This means that a corner 0 tile has 2 neighbors, a side 0 has 3 neighbors, and a center 0 has 4 neighbors.
                            kid = [x[:] for x in state] # makes a copy of the parent board
                            swap_positions(kid, row, index, neigh_row, neigh_index) # switches the 0 with its neighbor
                            kids.append(kid)
    return kids

# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    
    # IS IT THE CORRECT DIMENSIONS?
    sides = len(state) # every row should be as long as the entire puzzle is tall (in order for it to be a square)
    for row in state:
        if len(row) != sides: # if it's not a square
            return 0, 0, 0, None, -1
    
    # DOES IT HAVE THE RIGHT NUMBERS?
    # every board should have the numbers 0 to dimensions**2 - 1. So a 3x3 board will have 0-8, 4x4 will have 0-15, etc.
    ideal_list = list(range(sides**2)) # this is what the sorted board would look like
    flat_state = [item for sublist in state for item in sublist] # again, taken from https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
    sorted_list = sorted(flat_state) # now we have our board good and sorted
    if sorted_list != ideal_list: # are they the same? they should be.
        return 0, 0, 0, None, -1
    
    # IS IT SOLVABLE?
    # This method for checking for solvability was taken from https://www.geeksforgeeks.org/check-instance-15-puzzle-solvable/
    # However, the code is all my own for this check. Only the way of checking for solvability was taken from that source.
    '''
    The idea of this method is checking for "inversions" in the grid; i.e. when a tile is greater than following tiles.
    So if our board looks like [[0,1],[3,2]], we would have 1 inversion, the one of 3,2.
    The reason we need to know about inversions is because of the way the number of inversions when we make a move.
    A horizontal move will never change the number of inversions (switching the empty tile, which we dont consider,
    with any other tile).
    On a board of odd dimensions, if we were to transpose a vertical move from a normal board to a flattened board,
    the tile in question would pass an even number of tiles. And so the invertedness relationship between that tile
    and those even number of tiles will automatically flip, meaning the total number of inversions will not change
    even to odd. This means that for a board with odd dimensions, there must be an even number of inversions for it
    to be solvable.
    For a board with even dimensions, we are concerned about the same idea but now a vertical move will move the tile
    past an odd number of tiles (a horizontal move will still not change anything). However, we know that we need the
    0 tile to end up in the first row. So if the 0 tile is starting in the second row (or any even row from the top),
    the number of inversions must start odd. And if the 0 tile starts in the top row or any odd row, the number of
    inversions must be even.
    '''
    
    flat_state = [item for sublist in state for item in sublist] # again, taken from https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
    inversions = 0
    # Are the dimensions even or odd?
    if sides%2 != 0: # the dimensions are odd
        for index in range(len(flat_state)):
            if flat_state[index] != 0: # we don't want to count inversions involving the empty tile
                for later in range(index + 1, len(flat_state)):
                    if flat_state[later] != 0: # again, no empty inversions
                        if flat_state[index] > flat_state[later]: # if the earlier tile is greater than the later tile
                            inversions += 1 # that's an inversion
        if inversions % 2 != 0: # we want even inversions
            return 0, 0, 0, None, -2
    else: # the dimensions are even
        for index in range(len(flat_state)):
            if flat_state[index] != 0: # again, no empty inversions
                for later in range(index + 1, len(flat_state)):
                    if flat_state[later] != 0: # again, no empty inversions
                        if flat_state[index] > flat_state[later]: # if the earlier tile is greater than the later tile
                            inversions += 1 # that's an inversions

        for row in range(sides): # calculating distance from bottom of puzzle
            if 0 in state[row]:
                dist = sides - row
        if inversions % 2 != 0: # number of inversions is odd    
            if dist % 2 == 0: # 0 is on even row from bottom (odd row from top)
                return 0, 0, 0, None, -2
        else: # number of inversions is even
            if dist % 2 == 1: # 0 is on odd row from bottom (even row from top)
                return 0, 0, 0, None, -2
    
    # SOLVING THE PUZZLE
    '''
    1. Establish the root node, the goal state, and our visited and frontier lists
    2. Start with the first node in the frontier (in this case, our root node). Pop that node, removing it
    from the frontier.
    3. Check if that node is the goal state. Because of the nature of heuristic search (A* is a variation),
    the first goal state we find has an optimal path, so we can end there.
    If the node is not the goal:
    4. Check if we have visited it. If we have, check if the previous g-score is better
    than the current one (i.e. if it took fewer steps to get to the same state last time).
    If the previous one is better, skip this node since we have visited it before more quickly.
    If the g-score is worse in the visited list, then update the g-score for this state in the
    visited list and continue with this node (although we have visited it before, we are interested
    in finding a better path through this node).
    5. Get its kids and add them as new nodes. Add them to the frontier list. Sort the frontier list
    so that the nodes with the best f-scores are expanded first.
    6. Repeat.
    '''
    
    # 1. Establish the root node, the goal state, and our visited and frontier lists
    start = PuzzleNode(state, -1, heuristic(state), [])
    
    goal = np.reshape(list(range(sides**2)), (sides, sides))
    goal = list(list(i) for i in goal)
    
    visited = {}
    frontier = [[start.f_score, start]]
    # 2. Start with the first node in the frontier (in this case, our root node).
    
    steps = 0
    exp = 0
    max_frontier = 0
    opt_path = 0
    
    while frontier:
        max_frontier = max(max_frontier, len(frontier))
        cont = True
        [cur_f, cur_node] = frontier.pop(0) # Pop that first node, removing it from the frontier.
        exp += 1
        if cur_node.state == goal:
            # 3. Check if that node is the goal state. Because of the nature of heuristic search (A* is a variation),
            # the first goal state we find has an optimal path, so we can end there.
            opt_path = cur_node.path
            steps = len(opt_path) - 1
            return steps, exp, max_frontier, opt_path, _
        
        # If the node is not the goal:
        if str(cur_node.state) in visited.keys():
            # 4. Check if we have visited it. If we have, check if the previous g-score is better
            # than the current one (i.e. if it took fewer steps to get to the same state last time).
            # If the previous one is better, skip this node since we have visited it before more quickly.
            # If the g-score is worse in the visited list, then update the g-score for this state in the
            # visited list and continue with this node (although we have visited it before, we are interested
            # in finding a better path through this node).
            if visited[str(cur_node.state)] > cur_node.g_score:
                visited[str(cur_node.state)] = cur_node.g_score
            else:
                cont = False
        
        if cont == True:
            # 5. Get its kids and add them as new nodes. Add them to the frontier list.
            kids = get_kids(cur_node.state)
            for kid in kids:
                new_node = PuzzleNode(kid, cur_node.f_score, heuristic(kid), copy.deepcopy(cur_node.path))
                frontier.append([new_node.f_score, new_node])
            visited[str(cur_node.state)] = cur_node.g_score
        # Sort the frontier list so that the nodes with the best f-scores are expanded first.
        frontier.sort(key = lambda x: x[0]) # this line taken from https://stackoverflow.com/questions/17555218/python-how-to-sort-a-list-of-lists-by-the-fourth-element-in-each-list
        # Repeat.
    return 'fail', 'fail', 'fail', 'fail', 'fail'

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [214]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [215]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


The next cell didn't have time to run so it still says AssertionError but hopefully that is no longer the case.

In [216]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


AssertionError: 

In [213]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [196]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission